# Water flows downstream, so does information

This notebook, the data and the code can be downloaded [here](https://github.com/davidbrochart/hydro_forward_backward).

In this notebook I will show that the measure of the streamflow at a point on a river can be transfered to any point on a river of the same hydrological basin, not only downstream but also upstream. It comes from the fact that a part of the water present in the streamflow of a river comes from other rivers that flow into it. This hierarchical relationship can be used by algorithms to backpropagate the streamflow signal. Possible applications can be found in situations where available data is not sufficient to unambiguously calibrate a model, for instance when sparse measurements of precipitation or streamflow are available, or where the quality of these measurements is not high enough, as it is the case for satellite-based estimates.

## Introduction

A hydrological network in a basin is of hierarchical nature. As we can see in the figure below, lots of small streams flow into bigger ones, which get bigger and bigger as they collect more flows (while also being less and less numerous), and so on until only one river remains and reaches the basin's outlet.

In [ ]:
from IPython.display import Image
Image("../data/hydrosheds_amazon_large.jpg")

If we want to represent the structure of a hydrological network, we get a tree where the trunc is the main river at the outlet of the basin, and the leaves are the streams from upland areas (see figure below). This hierarchical nature, where a node's streamflow depends on the streamflow of its children (not only in space but also in time), has some interesting characteristics that the [forward-backward algorithm](https://en.wikipedia.org/wiki/Forward%E2%80%93backward_algorithm) can take advante of.

In [ ]:
Image("../data/stream-order.gif")

We will limit the scope of this study to the simplest form of network - a linear one, where we have only one river that flows through several subbasins. In the figure below, we denote these subbasins by *b<sub>n</sub>*, while the basins that include the upstream subbasins are called *B<sub>n</sub>* and defined by:

*B<sub>0</sub> = {b<sub>0</sub>}*

*B<sub>1</sub> = {b<sub>0</sub>, b<sub>1</sub>}*

*B<sub>2</sub> = {b<sub>0</sub>, b<sub>1</sub>, b<sub>2</sub>}*

*B<sub>3</sub> = {b<sub>0</sub>, b<sub>1</sub>, b<sub>2</sub>, b<sub>3</sub>}*

*B<sub>4</sub> = {b<sub>0</sub>, b<sub>1</sub>, b<sub>2</sub>, b<sub>3</sub>, b<sub>4</sub>}*

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.sankey import Sankey
fig = plt.figure(figsize=(16.9, 5))
ax = fig.add_subplot(1, 1, 1, xticks=[], yticks=[])
sankey = Sankey(ax=ax, unit=None)
sankey.add(flows=[1, -1], trunklength=2, patchlabel='$b_0$')
ws_nb = 5
for i in range(1, ws_nb):
    sankey.add(flows=[1, -1], trunklength=2, patchlabel=f'$b_{i}$', prior=i-1, connect=(1, 0))
sankey.finish()
plt.show()

In [ ]:
import numpy as np
from pandas import DataFrame
import sys
sys.path.append('../py')
from models import gr4j, delay
from misc import get_kde, uniform_density, lnprob_from_density
import mcmc

Let's first load some precipitation data. It doesn't matter very much where it comes from, it just looks like a plausible precipitation time series.

In [ ]:
p = np.load('../data/p.npy')

We will take a constant potential evapotranspiration, even though this is not very plausible, but it is not very important here.

In [ ]:
df = DataFrame()
df['p'] = p
df['e'] = 4

In order to generate synthetic streamflow data, we choose quite an arbitrary hydrological model, and feed it with the precipitation and potential evapotranspiration data.

In [ ]:
g = gr4j([1000, 0, 100, 10])
df['q_true_0'] = g.run([df['p'].values, df['e'].values])

This is for our basin *B<sub>0</sub>*. Now we want the stream at its outlet to flow into another subbasin, which itself flows into another subbasin, etc. We will suppose that all subbasins have the same area, and that the time it takes for the water to flow from the outlet of a subbasin to the outlet of the next one is the same for every subbasin, and can be described as a simple delay in the streamflow signal. The precipitation and potential evapotranspiration are uniform across all subbasins. From this we can compute the streamflow at the outlet of a basin recursively from the streamflow at the outlet of its upstream basin.

In [ ]:
delay_days = 1
d = delay(delay_days)
for i in range(1, ws_nb):
    area_head = i
    area_tail = 1
    df[f'q_true_{i}'] = (d.run(df[f'q_true_{i-1}'].values) * area_head + df['q_true_0'].values * area_tail) / (area_head + area_tail)

We want to remove information from our synthetic streamflows. This can be done by inserting missing values and by adding noise. It is also similar to the type of data we can get from satellite estimates. The noise is chosen to be Gaussian around the true value, and the remaining data is spread in time accross the subbasins.

In [ ]:
downsample = 30 # keep one data every 30 days for each subbasin
offset = downsample // ws_nb
for i in range(ws_nb):
    q_min = np.nanmin(df[f'q_true_{i}'])
    q_max = np.nanmax(df[f'q_true_{i}'])
    # noise is gaussian around true value
    std = (q_max - q_min) / 10
    q_noised = np.random.normal(df[f'q_true_{i}'], std, df['p'].values.size)
    q_noised = np.where(q_noised < 0, 0, q_noised)
    q = np.ones_like(q_noised) * np.nan
    q[offset*i::downsample] = q_noised[offset*i::downsample]
    df[f'q{i}'] = q

The figure below shows that the information we have about the streamflow is spread accross the subbasins, not only in space but also in time. The remaining of this study presents the forward-backward algorithm, which is well suited for taking advantage of such distributed information.

In [ ]:
columns = {f'q{i}':f'$Q_{i}$' for i in range(ws_nb)}
columns.update({'e': '$E$', 'p': '$P$'})
df[['p', 'e']].tail(2*360).rename(columns=columns).plot(figsize=(16.7, 5))
for i in range(ws_nb):
    plt.scatter(df.tail(2*360).index, df[f'q{i}'].tail(2*360).rename(columns=columns), label=f'$Q_{i}$')
plt.legend()
plt.title('Spread streamflow information accross subbasins')
plt.show()

# Forward

The first step is to pass the information forward (i.e. downstream), just like water flows. We start by calibrating *B<sub>0</sub>* as we usually do, using the measurements (*P, E, Q<sub>0</sub>*) over it. This gives us some information about the model parameters, but they might not be fully known at this point because of the poor measurements available. The idea is then to use a model of *B<sub>1</sub>* that makes use of *B<sub>0</sub>* and *b<sub>1</sub>*'s models, and to calibrate this dual model with the measurements over *B<sub>1</sub>*. Note that the dual model consists of twice as many parameters as a single model (plus one parameter for the delay), which could be an issue if we didn't have any prior information about these parameters. But it is not the case: the calibration of *B<sub>0</sub>*'s model helped us know its parameters better. When this information is used during the calibration of *B<sub>1</sub>*'s model, it constraints the possible values of the parameters of *b<sub>1</sub>*'s model. So information has passed from *B<sub>0</sub>* to *b<sub>1</sub>*, and if we repeat this process for *B<sub>2</sub>*, and so on until the outlet, we effectively make the information flow downstream.

Note that the basic mechanism in which information passes between two basins is the calibration of a dual model, i.e. a model that splits a basin in two submodels. It is through the calibration of this dual model that information can be transfered between the parameters of the two submodels, and consequently between the streamflows simulated by the submodels.

Let's first get the posterior probability distribution for the model parameters of *B<sub>0</sub>*, given a streamflow *Q<sub>0</sub>*. We take a uniform distribution for the prior of these parameters, because we don't know anything about them.

In [ ]:
def get_lnprob(peq, warmup):
    # a closure to have peq and warmup already known in lnprob
    def lnprob(x):
        q_obs = peq.q.values
        g = gr4j(x)
        q_sim = g.run([peq.p.values, peq.e.values])
        # remove warm-up data and missing values
        df = DataFrame({'q_sim': q_sim, 'q_obs': q_obs})[warmup:].dropna()
        # standard deviation = confidence we have in the observed data
        std = (np.nanmax(q_obs) - np.nanmin(q_obs)) / 10
        # error on observed streamflow is a gaussian with standard deviation std
        return -np.sum(np.square(df.q_sim.values - df.q_obs.values)) / (2 * std * std) - np.log(np.sqrt(2 * np.pi * std * std))
    return lnprob

peq = df[['p', 'e']]
peq['q'] = df['q0']
warmup = 365 * 2 # two years for the warmup of the model
lnprob = get_lnprob(peq, warmup)

In [ ]:
x0 = [1000, 0, 100, 10] # initial parameter values
sampler = mcmc.Sampler(x0, lnprob)
sampler.run(100) # discard MCMC burn-in samples
samples = sampler.run(1000)

The figure below shows the posterior probability distribution of *B<sub>0</sub>*'s model parameters (using Kernel Density Estimation). We can see that they are more or less centered on their true value, with some uncertainty.

In [ ]:
def plot_dist(dist, x0):
    x_nb = len(dist)
    f, ax = plt.subplots(1, x_nb)
    f.set_figwidth(x_nb * 4)
    for i in range(x_nb):
        x, y = dist[i]
        ax[i].plot(x, y)
        ax[i].axvline(x0[i], color='r', alpha=0.3)
        ax[i].set_title(f'$X_{i+1}$')
    plt.show()

In [ ]:
x_b0 = [get_kde(samples[:, i]) for i in range(4)]
plot_dist(x_b0, x0)

For *B<sub>1</sub>*, with discharge *Q<sub>1</sub>*, we need a model that consists of the models of *B<sub>0</sub>* (with its propagation delay) and *b<sub>1</sub>*. Thus the *lnprob* function has to be a little more complex. Also, it has to take into account the prior information on the parameters of *B<sub>0</sub>*'s model.

In [ ]:
def get_lnprob(peq, warmup, lnprob_prior):
    def lnprob(x):
        # prior
        lnp = 0
        for i, v in enumerate(x):
            lnp += lnprob_prior[i](v)
        if not np.isfinite(lnp):
            return -np.inf, np.ones_like(peq.p.values) * np.inf

        q_obs = peq.q.values
        std = (np.nanmax(q_obs) - np.nanmin(q_obs)) / 10
        x_head = x[:5]
        g_head = gr4j(x_head)
        x_tail = x[5:]
        g_tail = gr4j(x_tail)
        q_sim = (g_head.run([peq.p.values, peq.e.values]) + g_tail.run([peq.p.values, peq.e.values])) / 2
        df = DataFrame({'q_sim': q_sim, 'q_obs': q_obs})[warmup:].dropna()
        return lnp - np.sum(np.square(df.q_sim.values - df.q_obs.values)) / (2 * std * std) - np.log(np.sqrt(2 * np.pi * std * std)), q_sim
    return lnprob

peq = df[['p', 'e']]
peq['q'] = df['q1']
# prior probability distribution is uniform for subbasin b1
x_b1 = [uniform_density(1e-3, 1e4), uniform_density(-1, 1), uniform_density(1e-3, 1e3), uniform_density(1e-3, 1e3)]
d_prior = uniform_density(0, 1e3)
x_prior = x_b0 + [d_prior] + x_b1
lnprob_prior = [lnprob_from_density(p) for p in x_prior]
lnprob = get_lnprob(peq, warmup, lnprob_prior)

In [ ]:
x0 = [1000, 0, 100, 10] + [1] + [1000, 0, 100, 10]
sampler = mcmc.Sampler(x0, lnprob)
sampler.run(100)
samples, q_sim = sampler.run(1000)

From the figure below, where the posterior probability distribution of *B<sub>0</sub>*'s model parameters is shown before (lighter blue) and after (darker blue) the update, we can see that our knowledge of the parameters has changed, based on the information present in *Q<sub>1</sub>*. Whether the parameters have converged towards their true value or not depends on how much information was added by the new observed data.

In [ ]:
x_nb = 5
f, ax = plt.subplots(1, x_nb)
f.set_figwidth(x_nb * 3.2)
for i in range(x_nb):
    x, y = get_kde(samples[:, i])
    if i < 4:
        p = ax[i].plot(*x_b0[i], alpha=0.1)
    ax[i].plot(x, y, color=p[0].get_color())
    ax[i].axvline(x0[i], color='r', alpha=0.3)
    ax[i].set_title(f'$X_{i+1}$')
plt.show()

The figure below shows the posterior probability distribution for the parameters of *b<sub>1</sub>*'s model. Information from *B<sub>0</sub>*'s model parameters was used.

In [ ]:
x_b1 = [get_kde(samples[:, i]) for i in range(5, 9)]
plot_dist(x_b1, x0)

Now if we want to continue this process downstream, we could just create a model which includes the models of all its upstream basins. But that would not scale, first from a computational point of view, because the MCMC algorithm needs to run *(N models) * (M parameters)* for each sample, and then from a problem complexity point of view, because each time we add a parameter we add a dimension to the space that the MCMC algorithm has to explore. Even if the prior information we have on the parameters limits this space, it is always very important to limit the number of parameters for the algorithm to converve.

To prevent the problem complexity from growing, we can reduce our dual model to a single model. Indeed, we should now have a dual model that can produce (i.e. simulate) streamflow data with a better quality than the measured streamflow, since we also used the information in the streamflow of its upstream basin. All we need to do is to get the posterior probability distribution for the parameters of this single model using the simulated streamflow from the dual model. Fortunately, we already computed the streamflow of the dual model previously when we ran the MCMC algorithm. From this ensemble of streamflow time series, we get the mean and the standard deviation at each time step (considering that the distribution of the simulated streamflow is a Gaussian), as shown in the figure below.

In [ ]:
q_sim = np.array(q_sim)
q_sim[q_sim==np.inf] = np.nan
q_mean = np.nanmean(q_sim, axis=0)
q_std = np.nanstd(q_sim, axis=0)
q_std = np.where(q_std==0, np.min(q_std[np.nonzero(q_std)]), q_std)

In [ ]:
def plot_series(q_mean, q_std, q_true, q_meas, title):
    q_low = q_mean - q_std
    q_low = np.where(q_low < 0, 0, q_low)
    q_high = q_mean + q_std
    plt.figure(figsize=(16.6, 5))
    plt.plot(q_mean, label='Simulated streamfllow')
    plt.fill_between(np.arange(q_mean.size), q_low, q_high, color='r', alpha=0.2, label='Simulation uncertainty')
    plt.plot(q_true, color='b', linestyle='dashed', alpha=0.3, label='True streamflow')
    plt.scatter(np.arange(q_mean.size), q_meas, label='Measured streamflow')
    plt.title(title)
    plt.legend()
    plt.show()

In [ ]:
days = 2 * 365 # only plot two last years
q_mean_plot = q_mean[-days:]
q_std_plot = q_std[-days:] * 3 # show uncertainty between +/- 3 standard deviations
plot_series(q_mean_plot, q_std_plot, df[f'q_true_1'].values[-days:], df[f'q1'].values[-days:], 'Simulated streamflow of $B_1$ (dual model) using measured streamflows $Q_0$ and $Q_1$')

In [ ]:
def get_lnprob(peq, warmup):
    def lnprob(x):
        q_obs = peq.q_mean.values
        g = gr4j(x)
        q_sim = g.run([peq.p.values, peq.e.values])
        q_std = peq.q_std.values
        df = DataFrame({'q_sim': q_sim, 'q_obs': q_obs, 'q_std': q_std})[warmup:].dropna()
        std2 = df.q_std.values * df.q_std.values
        return -np.sum(np.square(df.q_sim.values - df.q_obs.values) / (2 * std2) - np.log(np.sqrt(2 * np.pi * std2))), q_sim
    return lnprob

peq = df[['p', 'e']]
peq['q_mean'] = q_mean
peq['q_std'] = q_std
lnprob = get_lnprob(peq, warmup)

In [ ]:
x0 = [1000, 0, 100, 10]
sampler = mcmc.Sampler(x0, lnprob)
sampler.run(100)
samples, q_sim = sampler.run(1000)

In [ ]:
q_sim = np.array(q_sim)
q_sim[q_sim==np.inf] = np.nan
q_mean = np.nanmean(q_sim, axis=0)
q_std = np.nanstd(q_sim, axis=0)
q_std = np.where(q_std==0, np.min(q_std[np.nonzero(q_std)]), q_std)

q_mean_plot = q_mean[-days:]
q_std_plot = q_std[-days:] * 3
plot_series(q_mean_plot, q_std_plot, df[f'q_true_1'].values[-days:], df[f'q1'].values[-days:], 'Simulated streamflow of $B_1$ (single model) using measured streamflows $Q_0$ and $Q_1$')

And now we are back in the situation we started in: we can create a dual model that consists of the models of *B<sub>1</sub>* and *b<sub>2</sub>*, etc. The following is the full algorithm which corresponds to the forward pass.

In [ ]:
def get_lnprob(peq, warmup, lnprob_prior, area_head, area_tail):
    def lnprob(x):
        lnp = 0
        for i, v in enumerate(x):
            lnp += lnprob_prior[i](v)
        if not np.isfinite(lnp):
            return -np.inf, np.ones_like(peq.p.values) * np.inf
        q_obs = peq.q_obs.values
        q_std = peq.q_std.values
        x_head = x[:5]
        g_head = gr4j(x_head)
        if area_tail > 0:
            x_tail = x[5:]
            g_tail = gr4j(x_tail)
            q_tail = g_tail.run([peq.p.values, peq.e.values])
        else:
            q_tail = 0
        q_sim = (g_head.run([peq.p.values, peq.e.values]) * area_head + q_tail * area_tail) / (area_head + area_tail)
        df = DataFrame({'q_sim': q_sim, 'q_obs': q_obs, 'q_std': q_std})[warmup:].dropna()
        std2 = df.q_std.values * df.q_std.values
        return lnp - np.sum(np.square(df.q_sim.values - df.q_obs.values) / (2 * std2) - np.log(np.sqrt(2 * np.pi * std2))), q_sim
    return lnprob

q_ensemble = {}
for ws_i in range(ws_nb):
    peq = df[['p', 'e']]
    peq['q_obs'] = df[f'q{ws_i}']
    peq['q_std'] = (np.nanmax(peq.q_obs) - np.nanmin(peq.q_obs)) / 10
    warmup = 365 * 2
    if ws_i == 0:
        area_head = 1
        area_tail = 0
        # prior probability distribution is uniform for head basin
        x_head = [uniform_density(1e-6, 1e6), uniform_density(-1e6, 1e6), uniform_density(1e-6, 1e6), uniform_density(1e-6, 1e6)]
        x0 = [1000, 0, 100, 10]
    x_prior = x_head
    if ws_i > 0:
        area_head = ws_i
        area_tail = 1
        x0 = [xy[0][np.argmax(xy[1])] for xy in x_head]
        x0 += [10] + [1000, 0, 100, 10]
        # prior probability distribution is uniform for tail basin
        d_prior = uniform_density(0, 1e2)
        x_tail = [uniform_density(1e-3, 1e4), uniform_density(-1, 1), uniform_density(1e-3, 1e3), uniform_density(1e-3, 1e2)]
        x_prior += [d_prior] + x_tail
    lnprob_prior = [lnprob_from_density(p) for p in x_prior]
    lnprob = get_lnprob(peq, warmup, lnprob_prior, area_head=area_head, area_tail=area_tail)
    # run MCMC
    sampler = mcmc.Sampler(x0, lnprob)
    sampler.run(100) # discard burn-in samples
    samples, q_sim = sampler.run(1000)
    # get simulated streamflow and uncertainty
    q_sim = np.array(q_sim)
    q_sim[q_sim==np.inf] = np.nan
    q_mean = np.nanmean(q_sim, axis=0)
    q_std = np.nanstd(q_sim, axis=0)
    q_std = np.where(q_std==0, np.min(q_std[np.nonzero(q_std)]), q_std)
    df[f'q_fmean{ws_i}'] = q_mean
    df[f'q_fstd{ws_i}'] = q_std
    q_ensemble[f'f{ws_i}'] = q_sim
    # plot updated streamflow
    q_mean_plot = q_mean[-days:]
    q_std_plot = q_std[-days:] * 3
    plot_series(q_mean_plot, q_std_plot, df[f'q_true_{ws_i}'].values[-days:], df[f'q{ws_i}'].values[-days:], f'Updated streamflow at the outlet of $B_{ws_i}$')
    if (ws_i > 0) and (ws_i < ws_nb - 1):
        # reduce dual model to single model
        peq = df[['p', 'e']]
        peq['q_obs'] = q_mean
        peq['q_std'] = q_std
        warmup = 365 * 2
        x_prior = [uniform_density(1e-6, 1e6), uniform_density(-1e6, 1e6), uniform_density(1e-6, 1e6), uniform_density(1e-6, 1e6)]
        lnprob_prior = [lnprob_from_density(p) for p in x_prior]
        lnprob = get_lnprob(peq, warmup, lnprob_prior, 1, 0)
        x0 = [1000, 0, 100, 10]
        sampler = mcmc.Sampler(x0, lnprob)
        sampler.run(100)
        samples, q_sim = sampler.run(1000)
    x_nb = 4
    x_head = [get_kde(samples[:, i]) for i in range(x_nb)]

# Backward

Passing the information forward results in the downstream basins benefiting from more information than the upstream basins. So for instance *Q<sup>4</sup>* is better known than *Q<sub>0</sub>* after the update. This doesn't have to be so, because we can feel that an information on a downstream basin can also constrain an upstream basin.

The second step is to pass the information backward (i.e. upstream). This is less intuitive since water doesn't flow upstream, but from the information point of view it works both ways: when we calibrate *b<sub>1</sub>*'s model using *B<sub>0</sub>*'s model, we can say that the information in *Q<sub>0</sub>* constraints the possible values of *Q<sub>1</sub>* further (information passes forward), or we can equivalently say that the information in *Q<sub>1</sub>* constraints the possible values of *Q<sub>0</sub>* further (information passes backward). Now if instead of starting from the two first basins we start from the two last ones, and repeat this process by going upstream, we effectively make the information flow backward. The following is the full algorithm corresponding to the backward step.

In [ ]:
def get_lnprob(peq, warmup, lnprob_prior, area_head, area_tail):
    def lnprob(x):
        lnp = 0
        for i, v in enumerate(x):
            lnp += lnprob_prior[i](v)
        if not np.isfinite(lnp):
            return -np.inf, np.ones_like(peq.p.values) * np.inf
        q_obs = peq.q_obs.values
        q_std = peq.q_std.values
        x_head = x[:5]
        g_head = gr4j(x_head)
        if area_tail > 0:
            x_tail = x[5:]
            g_tail = gr4j(x_tail)
            q_tail = g_tail.run([peq.p.values, peq.e.values])
        else:
            q_tail = 0
        q_head = g_head.run([peq.p.values, peq.e.values])
        q_sim = (q_head * area_head + q_tail * area_tail) / (area_head + area_tail)
        df = DataFrame({'q_sim': q_sim, 'q_obs': q_obs, 'q_std': q_std})[warmup:].dropna()
        std2 = df.q_std.values * df.q_std.values
        return lnp - np.sum(np.square(df.q_sim.values - df.q_obs.values) / (2 * std2) - np.log(np.sqrt(2 * np.pi * std2))), q_head
    return lnprob

for ws_i in range(ws_nb-1, 0, -1):
    if ws_i == ws_nb - 1:
        nb = 2
    else:
        nb = 1
    for n in range(nb-1, -1, -1):
        peq = df[['p', 'e']]
        peq['q_obs'] = df[f'q{ws_i - 1 + n}']
        peq['q_std'] = (np.nanmax(peq.q_obs) - np.nanmin(peq.q_obs)) / 10
        warmup = 365 * 2
        x_prior = [uniform_density(1e-6, 1e6), uniform_density(-1e6, 1e6), uniform_density(1e-6, 1e6), uniform_density(1e-6, 1e6)]
        x0 = [1000, 0, 100, 10]
        lnprob_prior = [lnprob_from_density(p) for p in x_prior]
        lnprob = get_lnprob(peq, warmup, lnprob_prior, 1, 0)
        # run MCMC
        sampler = mcmc.Sampler(x0, lnprob)
        sampler.run(100) # discard burn-in samples
        samples, q_sim = sampler.run(1000)
        q_sim = np.array(q_sim)
        q_sim[q_sim==np.inf] = np.nan
        q_mean = np.nanmean(q_sim, axis=0)
        q_std = np.nanstd(q_sim, axis=0)
        q_std = np.where(q_std==0, np.min(q_std[np.nonzero(q_std)]), q_std)
        if n == 1:
            # last basin: no update
            df[f'q_bmean{ws_nb - 1}'] = q_mean
            df[f'q_bstd{ws_nb - 1}'] = q_std
            q_ensemble[f'b{ws_nb - 1}'] = q_sim
        else:
            x_head = [get_kde(samples[:, i]) for i in range(4)]

    peq = df[['p', 'e']]
    if ws_i == ws_nb - 1:
        peq['q_obs'] = df[f'q{ws_i}']
        peq['q_std'] = (np.nanmax(peq.q_obs) - np.nanmin(peq.q_obs)) / 10
    else:
        peq['q_obs'] = q_mean
        peq['q_std'] = q_std
    warmup = 365 * 2
    x_prior = x_head
    area_head = ws_i
    area_tail = 1
    x0 = [xy[0][np.argmax(xy[1])] for xy in x_head]
    x0 += [10] + [1000, 0, 100, 10]
    # prior probability distribution is uniform for tail basin
    d_prior = uniform_density(0, 1e2)
    x_tail = [uniform_density(1e-3, 1e4), uniform_density(-1, 1), uniform_density(1e-3, 1e3), uniform_density(1e-3, 1e2)]
    x_prior += [d_prior] + x_tail
    lnprob_prior = [lnprob_from_density(p) for p in x_prior]
    lnprob = get_lnprob(peq, warmup, lnprob_prior, area_head=area_head, area_tail=area_tail)
    # run MCMC
    sampler = mcmc.Sampler(x0, lnprob)
    sampler.run(100) # discard burn-in samples
    samples, q_sim = sampler.run(1000)
    # get simulated streamflow and uncertainty
    q_sim = np.array(q_sim)
    q_sim[q_sim==np.inf] = np.nan
    q_mean = np.nanmean(q_sim, axis=0)
    q_std = np.nanstd(q_sim, axis=0)
    q_std = np.where(q_std==0, np.min(q_std[np.nonzero(q_std)]), q_std)
    df[f'q_bmean{ws_i-1}'] = q_mean
    df[f'q_bstd{ws_i-1}'] = q_std
    q_ensemble[f'b{ws_i - 1}'] = q_sim
    # plot updated streamflow
    if ws_i == ws_nb - 1:
        nb = 2
    else:
        nb = 1
    for n in range(nb-1, -1, -1):
        q_mean_plot = df[f'q_bmean{ws_i-1+n}'].values[-days:]
        q_std_plot = df[f'q_bstd{ws_i-1+n}'].values[-days:] * 3
        plot_series(q_mean_plot, q_std_plot, df[f'q_true_{ws_i-1+n}'].values[-days:], df[f'q{ws_i-1+n}'].values[-days:], f'Updated streamflow at the outlet of $B_{ws_i-1+n}$')

## Smoothing

The final step is to merge the forward and the backward passes, since they are complementary in their use of information.

In [ ]:
for ws_i in range(ws_nb):
    q_sim = np.vstack((q_ensemble[f'f{ws_i}'], q_ensemble[f'b{ws_i}']))
    q_mean = np.nanmean(q_sim, axis=0)
    q_std = np.nanstd(q_sim, axis=0)
    q_std = np.where(q_std==0, np.min(q_std[np.nonzero(q_std)]), q_std)
    q_mean_plot = q_mean[-days:]
    q_std_plot = q_std[-days:] * 3
    plot_series(q_mean_plot, q_std_plot, df[f'q_true_{ws_i}'].values[-days:], df[f'q{ws_i}'].values[-days:], f'Updated streamflow at the outlet of $B_{ws_i}$')

## Conclusion

This notebook showed how the forward-backward algorithm can be applied to hydrology in the context of distributed information, e.g. when sparse streamflow measurements are spread over the hydrological network, and/or when these measurements are of low quality. The power of the bayesian framework can be leveraged to take advantage of this situation and pass the information downstream (forward) and upstream (backward), resulting in an efficient use of the overall information. Satellite data is well suited to this kind of processing due to its spatial nature and poor precision. In particular, satellite-based streamflow estimates derived from water level is a good candidate.